# Ejemplo 01: Chat Rápido con OpenAI SDK

Este cuaderno demuestra cómo usar el SDK de OpenAI con Microsoft Foundry Local para interacciones rápidas de chat.

## Descripción General

Este ejemplo muestra:
- Uso del SDK de Python de OpenAI con Foundry Local
- Manejo de configuraciones tanto de Azure OpenAI como de Foundry local
- Implementación de manejo adecuado de errores y estrategias de respaldo
- Uso de FoundryLocalManager para la gestión de servicios


## Requisitos previos

Asegúrate de tener:
1. Foundry Local instalado y en funcionamiento
2. Un modelo cargado (por ejemplo, `foundry model run phi-4-mini`)
3. Los paquetes de Python necesarios instalados

### Instalar dependencias


In [ ]:
# Install required packages if not already installed
!pip install openai foundry-local-sdk

## Importar Bibliotecas Requeridas


In [ ]:
import os
import sys
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## Configuración

Configura tu configuración. Puedes usar cualquiera de las siguientes opciones:
1. **Azure OpenAI** (basado en la nube)
2. **Foundry Local** (inferencia local)

### Opción 1: Configuración de Azure OpenAI


In [ ]:
# Azure OpenAI Configuration (uncomment and set your values)
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"

### Opción 2: Configuración Local de Foundry


In [ ]:
# Foundry Local Configuration
MODEL_ALIAS = "phi-4-mini"  # Change this to your preferred model
BASE_URL = "http://localhost:8000"  # Default Foundry Local URL
API_KEY = ""  # Usually not needed for local

## Inicialización del Cliente

Crea el cliente de OpenAI según tu configuración:


In [ ]:
def create_client():
    """Create and configure OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if azure_endpoint and azure_api_key:
        # Azure OpenAI path
        model = os.environ.get("MODEL", "your-deployment-name")
        client = OpenAI(
            base_url=f"{azure_endpoint}/openai",
            api_key=azure_api_key,
            default_query={"api-version": azure_api_version},
        )
        print(f"🌐 Using Azure OpenAI with model: {model}")
        return client, model
    
    # Foundry Local path
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            manager = FoundryLocalManager(MODEL_ALIAS)
            model_info = manager.get_model_info(MODEL_ALIAS)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            model = model_info.id
            print(f"🏠 Using Foundry Local SDK with model: {model}")
            return client, model
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{BASE_URL}/v1",
        api_key=API_KEY
    )
    model = MODEL_ALIAS
    print(f"🔧 Using manual configuration with model: {model}")
    return client, model

# Initialize the client
client, model = create_client()

## Ejemplo básico de chat

Probemos una interacción de chat sencilla:


In [ ]:
def chat_with_model(prompt, max_tokens=128):
    """Send a chat message to the model and get a response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test with a simple prompt
prompt = "Say hello from Foundry Local and explain what you are in one sentence."
print(f"👤 User: {prompt}")
print("\n🤖 Assistant:")
response = chat_with_model(prompt)
print(response)

## Sesión de Chat Interactiva

Prueba diferentes tipos de indicaciones para ver cómo responde el modelo:


In [ ]:
# Example prompts to try
example_prompts = [
    "What are the benefits of edge AI?",
    "Explain the difference between local and cloud AI inference.",
    "Write a short Python function to calculate the factorial of a number.",
    "What is Microsoft Foundry Local?"
]

for i, prompt in enumerate(example_prompts, 1):
    print(f"\n{'='*50}")
    print(f"Example {i}: {prompt}")
    print(f"{'='*50}")
    
    response = chat_with_model(prompt, max_tokens=200)
    print(response)

## Uso avanzado: Respuestas en streaming

Para respuestas más extensas, el streaming puede ofrecer una mejor experiencia de usuario:


In [ ]:
def chat_with_streaming(prompt, max_tokens=300):
    """Send a chat message with streaming response."""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        print("🤖 Assistant (streaming):")
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        return f"Error: {e}"

# Test streaming with a longer prompt
streaming_prompt = "Explain the advantages of running AI models locally on device versus in the cloud. Include privacy, latency, and cost considerations."
print(f"👤 User: {streaming_prompt}\n")
streaming_response = chat_with_streaming(streaming_prompt)

## Manejo de Errores y Diagnósticos

Verifiquemos el estado del servicio y los modelos disponibles:


In [ ]:
def check_service_health():
    """Check Foundry Local service health."""
    try:
        # Try to list available models
        models_response = client.models.list()
        models = [model.id for model in models_response.data]
        
        print("✅ Service is healthy")
        print(f"📋 Available models: {models}")
        print(f"🎯 Current model: {model}")
        
        if model in models:
            print("✅ Current model is available")
        else:
            print("⚠️ Current model may not be loaded")
            
    except Exception as e:
        print(f"❌ Service check failed: {e}")
        print("\n🔧 Troubleshooting tips:")
        print("1. Make sure Foundry Local is running")
        print("2. Check if a model is loaded: foundry service ps")
        print("3. Verify the endpoint URL is correct")

check_service_health()

## Prueba de Indicaciones Personalizadas

Utiliza la celda a continuación para probar tus propias indicaciones:


In [ ]:
# Enter your custom prompt here
custom_prompt = "Write a haiku about artificial intelligence running on edge devices."

print(f"👤 User: {custom_prompt}\n")
custom_response = chat_with_model(custom_prompt, max_tokens=100)
print(f"🤖 Assistant: {custom_response}")

## Resumen

Este cuaderno demostró:

1. **✅ Configuración del Cliente**: Cómo configurar el SDK de OpenAI con Foundry Local  
2. **✅ Chat Básico**: Interacciones simples de solicitud-respuesta  
3. **✅ Streaming**: Generación de respuestas en tiempo real  
4. **✅ Manejo de Errores**: Manejo robusto de errores y diagnósticos  
5. **✅ Salud del Servicio**: Verificación de disponibilidad del modelo y estado del servicio  

### Próximos Pasos

- **Ejemplo 02**: Integración avanzada del SDK con soporte para Azure OpenAI  
- **Ejemplo 04**: Construcción de una aplicación de chat con Chainlit  
- **Ejemplo 05**: Sistemas de orquestación multi-agente  
- **Ejemplo 06**: Enrutamiento inteligente de modelos  

### Beneficios Clave de Foundry Local

- 🔒 **Privacidad**: Los datos nunca salen de tu dispositivo  
- ⚡ **Velocidad**: Inferencia local con baja latencia  
- 💰 **Costo**: Sin costos de uso de API  
- 🔌 **Offline**: Funciona sin conexión a internet  
- 🛠️ **Compatibilidad**: API compatible con OpenAI  
